## Project description

You work for the OilyGiant mining company. Your task is to find the best place for a new well.

Steps to choose the location:
- Collect the oil well parameters in the selected region: oil quality and volume of reserves;
- Build a model for predicting the volume of reserves in the new wells;
- Pick the oil wells with the highest estimated values;
- Pick the region with the highest total profit for the selected oil wells.

You have data on oil samples from three regions. Parameters of each oil well in the region are already known. Build a model that will help to pick the region with the highest profit margin. Analyze potential profit and risks using the Bootstrapping technique.

## Data description

Geological exploration data for the three regions are stored in files:

- `id` — unique oil well identifier
- `f0, f1, f2` — three features of points (their specific meaning is unimportant, but the features themselves are significant)
- `product` — volume of reserves in the oil well (thousand barrels).

Conditions:
- Only linear regression is suitable for model training (the rest are not sufficiently predictable).
- When exploring the region, a study of 500 points is carried with picking the best 200 points for the profit calculation.
- The budget for development of 200 oil wells is 100 USD million.
- One barrel of raw materials brings 4.5 USD of revenue The revenue from one unit of product is 4,500 dollars (volume of reserves is in thousand barrels).
- After the risk evaluation, keep only the regions with the risk of losses lower than 2.5%. From the ones that fit the criteria, the region with the highest average profit should be selected.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Data loading and preparation

In [2]:
# Read data into dataframes with different regions
df_reg0 = pd.read_csv('datasets/geo_data_0.csv')
df_reg1 = pd.read_csv('datasets/geo_data_1.csv')
df_reg2 = pd.read_csv('datasets/geo_data_2.csv')

In [3]:
# Display information on region 0
df_reg0.info()
display(df_reg0.head())
display(df_reg0.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [4]:
# Display information on region 1
df_reg1.info()
display(df_reg1.head())
display(df_reg1.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [5]:
# Display information on region 2
df_reg2.info()
display(df_reg2.head())
display(df_reg2.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [6]:
# Split data into features and target
# Region 0
features_reg0 = df_reg0.drop(['id', 'product'], axis=1)
target_reg0 = df_reg0['product']

# Region 1
features_reg1 = df_reg1.drop(['id', 'product'], axis=1)
target_reg1 = df_reg1['product']

# Region 2
features_reg2 = df_reg2.drop(['id', 'product'], axis=1)
target_reg2 = df_reg2['product']

In [7]:
# Split data into training and validation set 75/25
# Region 0
features_train_reg0, features_valid_reg0, target_train_reg0, target_valid_reg0 = train_test_split(features_reg0, target_reg0, test_size=0.25, random_state=12345)

# Region 1
features_train_reg1, features_valid_reg1, target_train_reg1, target_valid_reg1 = train_test_split(features_reg1, target_reg1, test_size=0.25, random_state=12345)

# Region 2
features_train_reg2, features_valid_reg2, target_train_reg2, target_valid_reg2 = train_test_split(features_reg2, target_reg2, test_size=0.25, random_state=12345)

In [8]:
# Confirm training and validation set sizes
print('Features training set size:', features_train_reg0.shape)
print('Features validation set size:', features_valid_reg0.shape)
print('Target training set size:', target_train_reg0.shape)
print('Target validation set size:', target_valid_reg0.shape)

Features training set size: (75000, 3)
Features validation set size: (25000, 3)
Target training set size: (75000,)
Target validation set size: (25000,)


## Model training and evaluation

In [9]:
# Train linear regression on model and prediction on validation set
# Region 0
model_reg0 = LinearRegression()
model_reg0.fit(features_train_reg0, target_train_reg0)
predicted_valid_reg0 = model_reg0.predict(features_valid_reg0)

# Region 1
model_reg1 = LinearRegression()
model_reg1.fit(features_train_reg1, target_train_reg1)
predicted_valid_reg1 = model_reg1.predict(features_valid_reg1)

# Region 2
model_reg2 = LinearRegression()
model_reg2.fit(features_train_reg2, target_train_reg2)
predicted_valid_reg2 = model_reg2.predict(features_valid_reg2)

In [10]:
# Calculate average volume for predictions, and RMSE
# Region 0
rmse_reg0 = mean_squared_error(target_valid_reg0, predicted_valid_reg0, squared=False)
avgVol_reg0 = predicted_valid_reg0.mean()
print(f'Region 0: Average volume of predicted reserves: {avgVol_reg0}, RMSE: {rmse_reg0}')

# Region 1
rmse_reg1 = mean_squared_error(target_valid_reg1, predicted_valid_reg1, squared=False)
avgVol_reg1 = predicted_valid_reg1.mean()
print(f'Region 1: Average volume of predicted reserves: {avgVol_reg1}, RMSE: {rmse_reg1}')

# Region 2
rmse_reg2 = mean_squared_error(target_valid_reg2, predicted_valid_reg2, squared=False)
avgVol_reg2 = predicted_valid_reg2.mean()
print(f'Region 2: Average volume of predicted reserves: {avgVol_reg2}, RMSE: {rmse_reg2}')

Region 0: Average volume of predicted reserves: 92.59256778438035, RMSE: 37.5794217150813
Region 1: Average volume of predicted reserves: 68.728546895446, RMSE: 0.8930992867756166
Region 2: Average volume of predicted reserves: 94.96504596800492, RMSE: 40.02970873393434


In [11]:
# Sanity check rmse values using a constant prediction consisting of the mean values
prediction_constant_reg0 = pd.Series(avgVol_reg0, index=target_valid_reg0.index) # Region 0
prediction_constant_reg1 = pd.Series(avgVol_reg1, index=target_valid_reg1.index) # Region 1
prediction_constant_reg2 = pd.Series(avgVol_reg2, index=target_valid_reg2.index) # Region 2

In [12]:
# Calculate RMSE of the constant prediction
rmse_constant_reg0 = mean_squared_error(target_valid_reg0, prediction_constant_reg0, squared=False)
rmse_constant_reg1 = mean_squared_error(target_valid_reg1, prediction_constant_reg1, squared=False)
rmse_constant_reg2 = mean_squared_error(target_valid_reg2, prediction_constant_reg2, squared=False)

# Print the results
print(f'Region 0: constant prediction check rmse: {rmse_constant_reg0}')
print(f'Region 1: constant prediction check rmse: {rmse_constant_reg1}')
print(f'Region 2: constant prediction check rmse: {rmse_constant_reg2}')

Region 0: constant prediction check rmse: 44.28900927907034
Region 1: constant prediction check rmse: 46.02124524071516
Region 2: constant prediction check rmse: 44.902157099150436


The ranking in terms of average volume of predicted reserves from high to low is region 2 > region 1 > region 0. The determined RMSE is better when compared against a constant prediction of the average value, and in all cases, the predictions from the trained model has a lower RMSE. This means that the trained model has a positive degree of prediction, being closer to the actual target values. The predictions for region 1 suggest very low error with a RMSE of less than 1 indicating that the model is trained well for this data set.

## Profit calculations

In [13]:
n_oil_wells = 200 # development for 200 wells
budget = 100000000 # USD million for n_oil_wells budget
rev_per_product = 4500 # revenue from one unit of product USD
cost_per_well = budget / n_oil_wells 
vol_breakeven = cost_per_well / rev_per_product # average volume of reserve to breakeven
print(f'Average volume per well to breakeven: {vol_breakeven:.2f}, with costs per well: {cost_per_well}')

print('Average volume in region 0:', target_reg0.mean())
print('Average volume in region 1:', target_reg1.mean())
print('Average volume in region 2:', target_reg2.mean())

Average volume per well to breakeven: 111.11, with costs per well: 500000.0
Average volume in region 0: 92.49999999999974
Average volume in region 1: 68.82500000002561
Average volume in region 2: 95.00000000000041


Given the conditions of a budget of 100 million USD for the development of 200 oil wells, and that each unit of product brings 4500 dollars of revenue, this results in an average cost per well to be $500,000, and an average volume per well needing to be at least 111.11 in order to breakeven. When we look at the average volume in each region, these values are well below the average volume per well needed to breakeven. This means that one cannot randomly select 200 oil wells in a region and expect to make a profit. Rather, identifying the most profitable oil wells and in which region is necessary to potentially maximize profit.

In [14]:
# Function to calculate profit from target set given predictions
def profit(target, predictions):
    # Sort predicted volumes from highest to lowest
    pred_sorted = pd.Series(predictions, index=target.index).sort_values(ascending=False)
    
    # Take the corresponding top 200 predicted volumes to the actual product volume and calculate the total revenue
    target_revenue = target[pred_sorted[0:200].index].sum() * rev_per_product

    # Return profit
    return target_revenue - budget # Budget of 100,000,000 dollars

In [15]:
# For each validation set and region, calculate profit for the highest values of predictions
# Region 0
profit_reg0 = profit(target_valid_reg0, predicted_valid_reg0)
print(f'Region 0: Model prediction max profit: ${profit_reg0:.2f}')

# Region 1
profit_reg1 = profit(target_valid_reg1, predicted_valid_reg1)
print(f'Region 1: Model prediction max profit: ${profit_reg1:.2f}')

# Region 2
profit_reg2 = profit(target_valid_reg2, predicted_valid_reg2)
print(f'Region 2: Model prediction max profit: ${profit_reg2:.2f}')

Region 0: Model prediction max profit: $33208260.43
Region 1: Model prediction max profit: $24150866.97
Region 2: Model prediction max profit: $27103499.64


In the entire region, the top 200 highest predicted volume reserves for oil wells were determined and the potential max profit was calculated. Based on this, it would be suggested for oil wells' development to be done in <b>Region 0</b> as this has potentially the highest profit of close of 33 million dollars compared to all of the other regions.

In [16]:
# Calculate profit from a random sample of 500 points, with top 200 in each set

# Region 0
# Sample 500 points
features_valid_reg0_subsample = features_valid_reg0.sample(n=500, replace=True, random_state=12345)
target_valid_reg0_subsample = target_valid_reg0[features_valid_reg0_subsample.index]

# generate prediction on the samples
pred_valid_reg0_subsample = model_reg0.predict(features_valid_reg0_subsample)

# Calculate profit
prof0 = profit(target_valid_reg0_subsample, pred_valid_reg0_subsample)
print(f'Region 0: profit from randomly selected 500 points: ${prof0:.2f}')

# Region 1
features_valid_reg1_subsample = features_valid_reg1.sample(n=500, replace=True, random_state=12345)
target_valid_reg1_subsample = target_valid_reg1[features_valid_reg1_subsample.index]
pred_valid_reg1_subsample = model_reg1.predict(features_valid_reg1_subsample)
prof1 = profit(target_valid_reg1_subsample, pred_valid_reg1_subsample)
print(f'Region 1: profit from randomly selected 500 points: ${prof1:.2f}')

# Region 2
features_valid_reg2_subsample = features_valid_reg2.sample(n=500, replace=True, random_state=12345)
target_valid_reg2_subsample = target_valid_reg2[features_valid_reg2_subsample.index]
pred_valid_reg2_subsample = model_reg2.predict(features_valid_reg2_subsample)
prof2 = profit(target_valid_reg2_subsample, pred_valid_reg2_subsample)
print(f'Region 2: profit from randomly selected 500 points: ${prof2:.2f}')

Region 0: profit from randomly selected 500 points: $9886908.27
Region 1: profit from randomly selected 500 points: $5918851.87
Region 2: profit from randomly selected 500 points: $2283965.90


From one random measurement of 500 locations, the predicted best 200 wells indicate that Region 0 has the highest potential profit at close to 10 million dollars compared to the other regions.

## Distribution of profit per region

Process here is to use bootstrapping technique with 1000 samples to find the distribution of profit. From a selection of 500 points, the top 200 predicted volumes and the expected profits are determined. From the 1000 samples, we can determine the average profit, and construct a 95% confidence interval. A count is also constructed to determine the percentage where profits are negative (loss).

In [17]:
# Region 0
state = np.random.RandomState(12345)

# Store profit values
profit_dist_reg0 = []
count_loss_reg0 = 0
for i in range(1000):
    # sampling 500 points
    features_valid_reg0_subsample = features_valid_reg0.sample(n=500, replace=True, random_state=state)
    target_valid_reg0_subsample = target_valid_reg0[features_valid_reg0_subsample.index]
    # generate prediction on the samples
    pred_valid_reg0_subsample = model_reg0.predict(features_valid_reg0_subsample)
    # Calculate profit and add to profit values
    prof = profit(target_valid_reg0_subsample, pred_valid_reg0_subsample)
    profit_dist_reg0.append(prof)
    # If at loss, add to counter
    if prof < 0:
        count_loss_reg0 += 1

profit_reg0 = pd.Series(profit_dist_reg0) # convert to dataframe

avg_profit_reg0 = profit_reg0.mean()

# lower and upper quantiles for confidence interval 95%
profit_lower = profit_reg0.quantile(0.025)
profit_higher = profit_reg0.quantile(0.975)

# Determine percentage of loss
loss_percent_reg0 = count_loss_reg0 / len(profit_dist_reg0)

# Print out results
print(f'Region 0: Average profit: {avg_profit_reg0:.2f}')
print(f'95% confidence interval: {profit_lower:.2f} - {profit_higher:.2f}')
print(f'Risk of loss, percentage: {loss_percent_reg0:.2%}')

Region 0: Average profit: 6007352.44
95% confidence interval: 129483.31 - 12311636.06
Risk of loss, percentage: 2.00%


In [18]:
# Region 1

# Store profit values
profit_dist_reg1 = []
count_loss_reg1 = 0
for i in range(1000):
    # sampling 500 points
    features_valid_reg1_subsample = features_valid_reg1.sample(n=500, replace=True, random_state=state)
    target_valid_reg1_subsample = target_valid_reg1[features_valid_reg1_subsample.index]
    # generate prediction on the samples
    pred_valid_reg1_subsample = model_reg1.predict(features_valid_reg1_subsample)
    # Calculate profit and add to profit values
    prof = profit(target_valid_reg1_subsample, pred_valid_reg1_subsample)
    profit_dist_reg1.append(prof)
    # If at loss, add to counter
    if prof < 0:
        count_loss_reg1 += 1

profit_reg1 = pd.Series(profit_dist_reg1) # convert to dataframe

avg_profit_reg1 = profit_reg1.mean()

# lower and upper quantiles for confidence interval 95%
profit_lower = profit_reg1.quantile(0.025)
profit_higher = profit_reg1.quantile(0.975)

# Determine percentage of loss
loss_percent_reg1 = count_loss_reg1 / len(profit_dist_reg1)

# Print out results
print(f'Region 1: Average profit: {avg_profit_reg1:.2f}')
print(f'95% confidence interval: {profit_lower:.2f} - {profit_higher:.2f}')
print(f'Risk of loss, percentage: {loss_percent_reg1:.2%}')

Region 1: Average profit: 6639589.95
95% confidence interval: 2064763.61 - 11911976.85
Risk of loss, percentage: 0.10%


In [19]:
# Region 2

# Store profit values
profit_dist_reg2 = []
count_loss_reg2 = 0
for i in range(1000):
    # sampling 500 points
    features_valid_reg2_subsample = features_valid_reg2.sample(n=500, replace=True, random_state=state)
    target_valid_reg2_subsample = target_valid_reg2[features_valid_reg2_subsample.index]
    # generate prediction on the samples
    pred_valid_reg2_subsample = model_reg2.predict(features_valid_reg2_subsample)
    # Calculate profit and add to profit values
    prof = profit(target_valid_reg2_subsample, pred_valid_reg2_subsample)
    profit_dist_reg2.append(prof)
    # If at loss, add to counter
    if prof < 0:
        count_loss_reg2 += 1

profit_reg2 = pd.Series(profit_dist_reg2) # convert to dataframe

avg_profit_reg2 = profit_reg2.mean()

# lower and upper quantiles for confidence interval 95%
profit_lower = profit_reg2.quantile(0.025)
profit_higher = profit_reg2.quantile(0.975)

# Determine percentage of loss
loss_percent_reg2 = count_loss_reg2 / len(profit_dist_reg2)

# Print out results
print(f'Region 2: Average profit: {avg_profit_reg2:.2f}')
print(f'95% confidence interval: {profit_lower:.2f} - {profit_higher:.2f}')
print(f'Risk of loss, percentage: {loss_percent_reg2:.2%}')

Region 2: Average profit: 5973810.48
95% confidence interval: 17349.30 - 12462179.60
Risk of loss, percentage: 2.50%


## Results and recommendations

I recommend the choice of Region 1 for the development of the oil wells for the goal of maximizing profit.

From this analysis, the highest average profit for oil well development would be in Region 1. Region 1 has the highest average profit of 6.64 million dollars, while Region 0 is slightly lower at about 6 million dollars.

Not only does Region 1 have the highest average profit, the risk of losses is the lowest at 0.10%, compared to Region 0 at 2.0%. We can disregard Region 2 completely, because the calculated risk of loss is at 2.5%, which does not meet our criteria.